In [1]:
import numpy as np
import math
from numpy.linalg import inv, svd

In [26]:
 np.hstack((np.ones(4), 1))

array([ 1.,  1.,  1.,  1.,  1.])

In [15]:
np.vstack((X, np.ones(4)))

array([[ 2. ,  3. ,  4. ,  5. ],
       [ 2.2,  3.1,  4.2,  5.3],
       [ 2.1,  3.2,  4.1,  5.2],
       [ 1. ,  1. ,  1. ,  1. ]])

In [44]:
def fit(X, y, lmbda):
    U, s, V = svd(X, full_matrices = False)
    V = V.T
    Z = U.dot(np.diag(s))
    Sigma = np.cov(Z, rowvar = False, ddof = 0)
    Sigma = lmbda * np.diag(Sigma) + (1 - lmbda) * Sigma
    VInvSigma = V.dot(inv(Sigma))
    Betas = []
    cs = np.unique(y)
    
    for c in cs:
        i = y == c
        prior = np.mean(i)
        mu_c = np.mean(X[i], axis = 0)
        mu_z_c = V.T.dot(mu_c)
        Beta = VInvSigma.dot(mu_z_c)
        gamma = -mu_c.T.dot(Beta) / 2.0 + math.log(prior)
        Betas.append(np.hstack((Beta, gamma)))
        
    return cs, np.transpose(Betas)
        
def predict(model, X):
    X = np.hstack((X, np.ones(len(X)).reshape(-1, 1)))
    cs, Beta = model
    return np.array([cs[i] for i in np.argmax(X.dot(Beta), axis = 1)])

model = fit(X, y, 0.1)
np.mean(predict(model, X) == y)

0.33333333333333331

In [2]:
X = np.array([
    [2, 3, 4, 5],
    [2.2, 3.1, 4.2, 5.3],
    [2.1, 3.2, 4.1, 5.2],
])

y = np.array([1, 1, 2])

In [3]:
def fit(X, y, lmbda):
    theta = {}

    U, s, V = svd(X, full_matrices = False)
    V = V.T
    Z = U.dot(np.diag(s))
    Sigma = np.cov(Z, rowvar = False, ddof = 0)
    Sigma = lmbda * np.diag(Sigma) + (1 - lmbda) * Sigma
    VInvSigma = V.dot(inv(Sigma))
    
    for c in np.unique(y):
        i = y == c
        prior = np.mean(i)
        mu_c = np.mean(X[i], axis = 0)
        mu_z_c = V.T.dot(mu_c)
        Beta = VInvSigma.dot(mu_z_c)
        gamma = -mu_c.T.dot(Beta) / 2.0 + math.log(prior)
        theta[c] = Beta, gamma
        
    return theta
        
def predict(model, X):
    p = []
    cs = []
        
    for c, theta in model.iteritems():
        Beta, gamma = theta
        cs.append(c)
        p.append(X.dot(Beta) + gamma)
      
    return np.array([cs[i] for i in np.argmax(p, axis = 0)])

In [4]:
model = fit(X, y, 0.1)
np.mean(predict(model, X) == y)

0.33333333333333331